# 6. Purely functional state

## 6.1 Generating random numbers using side effects

In [78]:
val rng=new scala.util.Random


scala.util.Random@466942

In [79]:
rng.nextDouble

0.8446207737410353

In [80]:
rng.nextDouble

0.07825994102430411

In [81]:
rng.nextInt

1392889429

In [82]:
def rollingDie: Int = {
    val rng = new scala.util.Random
    rng.nextInt(6)
}

In [83]:
rollingDie

5

In [84]:
rollingDie

3

In [85]:
rollingDie

2

=> all using internal state variable => SIDE EFFECT!

## 6.2 Purely functional random number generation

In [86]:
trait RNG {
    def nextInt: (Int, RNG)
}

### => 난수 계산과 state 전달 분리.

In [87]:
case class SimpleRNG(seed: Long) extends RNG {
    def nextInt: (Int, RNG) = {
        val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL
        val nextRNG = SimpleRNG(newSeed)
        val n = (newSeed >>> 16).toInt
        (n, nextRNG)
    }
}

In [88]:
val rng = SimpleRNG(42)

SimpleRNG(42)

In [89]:
val (n1, rng2) = rng.nextInt

16159453

In [90]:
val (n2, rng3) = rng2.nextInt

-1281479697

In [91]:
rng3

SimpleRNG(197491923327988)

In [92]:
rng2

SimpleRNG(1059025964525)

In [93]:
rng

SimpleRNG(42)

## 6.3 Making stateful APIs pure

In [94]:
def randomPair0(rng:RNG):(Int, Int) = {
    val (i1, _) = rng.nextInt
    val (i2, _) = rng.nextInt
    (i1,i2)
}

In [95]:
randomPair0(rng2)

(-1281479697,-1281479697)

In [96]:
def randomPair(rng:RNG): ((Int,Int), RNG) = {
    val (i1, rng2) = rng.nextInt
    val (i2, rng3) = rng2.nextInt
    ((i1,i2),rng3)
}

In [97]:
randomPair(rng2)

((-1281479697,-340305902),SimpleRNG(259172689157871))

### Ex.6.1 nonNegativeInt [0 ~ Int.maxValue]

In [98]:
def nonNegativeInt(rng:RNG): (Int, RNG) = {
    var n = 0
    val (i, rngNext) = rng.nextInt
    if (i==Int.MinValue) n = - (Int.MinValue+1)
    if (i<0) n = -i
    else n = i
    (n,rngNext)
// book's answer
// if (i < 0) (-(i+1),rngNext) else (i, rngNext) 
}

In [99]:
val (n1, r3) = nonNegativeInt(rng2)

1281479697

In [100]:
r3

SimpleRNG(197491923327988)

In [101]:
nonNegativeInt(r3)

(340305902,SimpleRNG(259172689157871))

> As you write more functional programs, you’ll sometimes encounter situations like
this where the functional way of expressing a program feels awkward or tedious. Does
this imply that purity is the equivalent of trying to write an entire novel without using
the letter E? Of course not. ** Awkwardness like this is almost always a sign of some
missing abstraction waiting to be discovered.**

### Ex.6.2 double

In [147]:
def double(rng:RNG): (Double, RNG) = {
    val (i,r) = nonNegativeInt(rng)
    if (i!=Int.MaxValue) ((i.toDouble / Int.MaxValue.toDouble), r)
    else ((Int.MaxValue-1).toDouble / Int.MaxValue.toDouble, r)
/*** book's answer
def double(rng: RNG): (Double, RNG) = {
  val (i, r) = nonNegativeInt(rng)
  (i / (Int.MaxValue.toDouble + 1), r)
}
****/
}

In [148]:
double(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

In [149]:
def intDouble(rng:RNG): ((Int,Double), RNG) = {
    val (i, r1) = rng.nextInt
    val (d, r2) = double(r1)
    ((i,d), r2)
}

In [150]:
intDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

### Ex.6.3 doubleInt, intDouble, double3

In [151]:
def doubleInt(rng:RNG): ((Double,Int), RNG) = {
    val (i, r1) = rng.nextInt
    val (d, r2) = double(r1)
    ((d, i), r2)
}

In [152]:
doubleInt(rng2)

((0.15846728447753344,-1281479697),SimpleRNG(259172689157871))

In [153]:
def double3(rng:RNG): ((Double,Double,Double),RNG) = {
    val (d1, r1) = double(rng)
    val (d2, r2) = double(r1)
    val (d3, r3) = double(r2)
    ((d1,d2,d3),r3)
}

In [154]:
double3(rng2)

((0.5967354856416283,0.15846728447753344,0.9386595436086224),SimpleRNG(149370390209998))

### Ex.6.4 ints

In [155]:
def ints(count:Int)(rng:RNG): (List[Int], RNG) = {
    def go(n:Int,rl:List[Int],rngIn:RNG): (List[Int],RNG) = {
        if (n==0) (rl, rngIn)
        else {
            val (i,rngNext) = rngIn.nextInt
            go(n-1, rl :+ i, rngNext)
        }
    }
    go(count,List(),rng)
}

In [156]:
ints(5)(rng2)

(List(-1281479697, -340305902, -2015756020, 1770001318, -1934589059),SimpleRNG(154689748186168))

## 6.4 A better API for state actions

### common pattern("state actions", "state transitions"): 
        RNG => (A, RNG)

### RNG state action data type:

In [157]:
type Rand[+A] = RNG => (A, RNG)

#### nextInt as a type of Rand[Int]

In [158]:
val int: Rand[Int] = _.nextInt

<function1>

### combinators - unit (do nothing)

In [159]:
def unit[A](a:A): Rand[A] = 
    rng => (a,rng)

### combinators - map (function composition)

In [160]:
def map[A,B](s:Rand[A])(f: A => B): Rand[B] = 
    rng => {
        val (a, rng2) = s(rng)
        (f(a), rng2)
    }

#### using map to implement nonNegativeEven

In [161]:
def nonNegativeEven: Rand[Int] =
    map(nonNegativeInt)(i => i - i % 2)

In [162]:
nonNegativeEven(rng2)

(1281479696,SimpleRNG(197491923327988))

In [163]:
nonNegativeEven(rng3)

(340305902,SimpleRNG(259172689157871))

### Ex.6.5 double(using 'map'. cf. Ex.6.2)

In [164]:
def convToDouble(i:Int): Double = {
    val maxFloat = Int.MaxValue.toDouble
    if (i!=Int.MaxValue) i.toDouble / maxFloat
    else (i-1).toDouble / maxFloat
}
    
def doubleViaMap: Rand[Double] = 
    map(nonNegativeInt)(i => convToDouble(i))

In [165]:
doubleViaMap(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

In [166]:
double(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

### 6.4.1 Combining state actions

In [167]:
def map2[A,B,C](ra:Rand[A], rb:Rand[B])(f: (A,B) => C): Rand[C] = {
    rng => {
        val (a, rngA) = ra(rng)
        val (b, rngB) = rb(rngA)
        (f(a,b), rngB)
    }
}

In [168]:
def both[A,B](ra:Rand[A], rb:Rand[B]): Rand[(A,B)] = map2(ra,rb)((_, _))

In [169]:
val randIntDouble: Rand[(Int,Double)] = both(int,double)

<function1>

In [170]:
val randDoubleInt: Rand[(Double,Int)] = both(double, int)

<function1>

In [171]:
intDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

In [172]:
randIntDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

### Ex.6.7 sequence

In [173]:
def sequence[A](fs: List[Rand[A]]): Rand[List[A]] = {
    def go(ss: List[Rand[A]], ll:List[A], rngIn:RNG): (List[A], RNG) = ss match {
        case h :: t => {
            val (a, rngOut) = h(rngIn)
            go(t, ll :+ a, rngOut)
        }
        case _ => (ll, rngIn)
    }
    rng => go(fs, List(), rng)
/*
book's answer:
def sequence ... 
    fs.foldRight(unit(List[A]()))((f,acc) => map2(f,acc)(_ :: _))

def _ints(count:Int): Rand[List[Int]] = 
    sequence(List.fill(count)(int))
*/
}

In [174]:
def randInts(count:Int)(rng:RNG): (List[Int], RNG) = {
    sequence(List.fill(count)(int))(rng)
}

In [175]:
ints(5)(rng2)

(List(-1281479697, -340305902, -2015756020, 1770001318, -1934589059),SimpleRNG(154689748186168))

In [176]:
randInts(5)(rng2)

(List(-1281479697, -340305902, -2015756020, 1770001318, -1934589059),SimpleRNG(154689748186168))

### 6.4.2 Nesting state actions

In [177]:
def nonNegativeLessThan_1st_try(n:Int): Rand[Int] = 
    map(nonNegativeInt)(_ % n)

#### => not fair (or 'skewed') (Why?? cf. https://forums.manning.com/posts/list/34423.page)

In [205]:
def tryN(count:Int)(f: Rand[Int]): Rand[List[Int]] = sequence(List.fill(count)(f))

In [206]:
tryN(100)(nonNegativeLessThan_1st_try(100))(rng2)

(List(97, 2, 20, 18, 59, 12, 41, 59, 74, 94, 32, 17, 91, 81, 21, 62, 33, 77, 70, 0, 36, 48, 92, 59, 64, 63, 11, 99, 16, 76, 22, 87, 43, 24, 70, 2, 47, 39, 68, 22, 36, 2, 64, 78, 85, 22, 39, 69, 46, 39, 4, 4, 27, 36, 18, 11, 16, 36, 8, 9, 26, 74, 25, 94, 54, 52, 18, 53, 60, 22, 69, 49, 91, 2, 4, 0, 97, 19, 29, 98, 32, 55, 48, 6, 43, 19, 75, 84, 64, 42, 68, 13, 82, 35, 9, 20, 29, 6, 13, 60),SimpleRNG(186499461027745))

In [207]:
tryN(100)(nonNegativeLessThan_1st_try(100))(rng2)._1.filter(_ < 50).length

60

In [181]:
def nonNegativeLessThan_2nd(n: Int): Rand[Int] = { rng =>
    val (i, rng2) = nonNegativeInt(rng)
    val mod = i % n
    if (i + (n-1) - mod >= 0)
        (mod, rng2)
    else nonNegativeLessThan_2nd(n)(rng)
}

In [208]:
tryN(1000)(nonNegativeLessThan_1st_try(100))(SimpleRNG(-1)) == 
tryN(1000)(nonNegativeLessThan_2nd(100))(SimpleRNG(-1))

true

### Ex. 6.8 flatMap

In [184]:
def flatMap[A,B](f: Rand[A])(g: A => Rand[B]): Rand[B] = {
    rng => {
        val (i,rngNext) = f(rng)
        g(i)(rngNext)
    }
}

In [185]:
def nonNegativeLessThan(n:Int): Rand[Int] = {
    flatMap(nonNegativeInt){ i =>
        val mod = i % n
        if (i + (n-1) - mod >= 0) unit(mod) else nonNegativeLessThan(n)
    }
}

In [209]:
tryN(100)(nonNegativeLessThan(100))(rng2)._2

SimpleRNG(186499461027745)

### Ex. 6.9 mapViaFlatMap, map2ViaFlatMap

In [188]:
def mapViaFlatMap[A,B](s: Rand[A])(f: A => B): Rand[B] = {
    /* flatMap(s)(a => (f(a),_)) */
    flatMap(s)(a => unit(f(a)))
}

In [189]:
def _nonNegativeEven: Rand[Int] =
    mapViaFlatMap(nonNegativeInt)(i => i - i % 2)

In [190]:
nonNegativeEven(rng2)

(1281479696,SimpleRNG(197491923327988))

In [191]:
_nonNegativeEven(rng2)

(1281479696,SimpleRNG(197491923327988))

In [192]:
def map2ViaFlatMap[A,B,C](ra: Rand[A], rb:Rand[B])(f: (A,B) => C): Rand[C] = {
   flatMap(ra)(a => map(rb)(b => f(a,b)))
}

In [193]:
def _both[A,B](ra:Rand[A], rb:Rand[B]): Rand[(A,B)] = {
    map2ViaFlatMap(ra,rb)((_, _))
    /* def both[A,B](ra:Rand[A], rb:Rand[B]): Rand[(A,B)] = map2(ra,rb)((_, _)) */
}
val _randIntDouble: Rand[(Int,Double)] = _both(int,double)

<function1>

In [194]:
randIntDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

In [195]:
_randIntDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

In [196]:
def rollDie: Rand[Int] = nonNegativeLessThan(6)

In [210]:
tryN(100)(rollDie)(rng2)

(List(3, 2, 4, 4, 5, 2, 1, 5, 2, 4, 4, 1, 3, 1, 3, 4, 3, 3, 4, 0, 4, 0, 2, 1, 2, 3, 5, 5, 0, 2, 4, 5, 5, 2, 4, 4, 3, 1, 2, 2, 2, 2, 4, 2, 5, 4, 3, 1, 4, 1, 4, 4, 1, 0, 4, 5, 2, 0, 2, 5, 4, 4, 3, 2, 2, 0, 0, 3, 2, 0, 3, 1, 1, 2, 2, 4, 3, 1, 3, 0, 2, 3, 2, 2, 3, 1, 1, 4, 2, 4, 2, 5, 4, 5, 5, 2, 1, 4, 3, 4),SimpleRNG(186499461027745))

In [212]:
def _rollDie: Rand[Int] = map(nonNegativeLessThan(6))(_ + 1)

In [213]:
tryN(100)(_rollDie)(rng2)

(List(4, 3, 5, 5, 6, 3, 2, 6, 3, 5, 5, 2, 4, 2, 4, 5, 4, 4, 5, 1, 5, 1, 3, 2, 3, 4, 6, 6, 1, 3, 5, 6, 6, 3, 5, 5, 4, 2, 3, 3, 3, 3, 5, 3, 6, 5, 4, 2, 5, 2, 5, 5, 2, 1, 5, 6, 3, 1, 3, 6, 5, 5, 4, 3, 3, 1, 1, 4, 3, 1, 4, 2, 2, 3, 3, 5, 4, 2, 4, 1, 3, 4, 3, 3, 4, 2, 2, 5, 3, 5, 3, 6, 5, 6, 6, 3, 2, 5, 4, 5),SimpleRNG(186499461027745))

## 6.5 A general state action data type